# Assessment Factors

With this report we want to examine the assessment factors, and making it simpler to analyse by factors. 

In the incoming data schema, the factors are a list of text values:

```
<Assessments>
    <AssessmentActualStartDate>1970-06-03</AssessmentActualStartDate>
    <AssessmentInternalReviewDate>1970-06-22</AssessmentInternalReviewDate>
    <AssessmentAuthorisationDate>1971-07-18</AssessmentAuthorisationDate>
    <FactorsIdentifiedAtAssessment>
        <AssessmentFactors>2A</AssessmentFactors>
        <AssessmentFactors>2B</AssessmentFactors>
    </FactorsIdentifiedAtAssessment>
</Assessments>
```

The ingest tool converts these to a comma-separated list, e.g. "2A,2B".

To make it easier to analyse we want to convert these to 
[dummy-variables](https://en.wikipedia.org/wiki/Dummy_variable_(statistics)), or 
[one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) as it's often called. 

In the actual pipeline we expect the data to come in a very specific format, but as we really only need the factors column to make this work, we are not really concerned about the other columns.

First we create a bit of a messy dataset of the type we're used to. A mix of values and non-values. 

For CHILD1 we se there are two rows - we may be looking at a subset of the data and there may be a natural key that has been removed from this subset. 

CHILD2 tests resilience with blanks and empties, and CHILD3 has repeat factors. 

In [1]:
import pandas as pd
from liiatools.cin_census_pipeline.reports import expanded_assessment_factors

In [2]:
df = pd.DataFrame([
    ["CHILD1", "A,B,C"],
    ["CHILD1", "A,B"],
    ["CHILD2", "A"],
    ["CHILD2", "B, C"],
    ["CHILD2", None],
    ["CHILD2", ""],
    ["CHILD3", "D,A,D"]
], columns=["LAchildID", "Factors"])
df

,LAchildID,Factors
0,CHILD1,"A,B,C"
1,CHILD1,"A,B"
2,CHILD2,A
3,CHILD2,"B, C"
4,CHILD2,None
5,CHILD2,
6,CHILD3,"D,A,D"


Now, we are only interested in the Factors column - so let's just isolate that column. 

We can use `str.split` to convert the comma-separated values into a list.

In [3]:
factors = df[['Factors']].copy()
factors['Factors'] = factors['Factors'].str.split(",")
factors

,Factors
0,"[A, B, C]"
1,"[A, B]"
2,[A]
3,"[B, C]"
4,None
5,[]
6,"[D, A, D]"


Now, a few things to note. We have quite a lot of control over this dataset, but I have added some whitespace
to illustrate some of the issue we still may face. In row 3 we have whitespace between the "," and the "C". 

This will become important. 

In [4]:
factors_exploded = factors.explode('Factors')
factors_exploded

,Factors
0,A
0,B
0,C
1,A
1,B
2,A
3,B
3,C
4,None
5,


The `explode` method turns the list into individal rows. Notice how our index entries are now duplicated. We will use these index entries later to merge the final view back into the original dataset. 

We could now do `get_dummies` to get the dummy variables:

In [5]:
pd.get_dummies(factors_exploded, columns=['Factors'], prefix="", prefix_sep="")

,,C,A,B,C,D
0,0,0,1,0,0,0
0,0,0,0,1,0,0
0,0,0,0,0,1,0
1,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,0,0,0,0
5,1,0,0,0,0,0


However, we see that we get both a column with no label, and two Cs. This is in fact a column for the empty string "" entry as well as one standard "C" and one " C" from the row with "B, C". So before we convert to dummies, let's strip whitespace and remove empty strings:

In [6]:
factors_dummies = factors_exploded.copy()
factors_dummies['Factors'] = factors_dummies['Factors'].str.strip()
factors_dummies = factors_dummies[factors_dummies['Factors'] != '']

factors_dummies = pd.get_dummies(factors_dummies, columns=['Factors'], prefix="", prefix_sep="")
factors_dummies

,A,B,C,D
0,1,0,0,0
0,0,1,0,0
0,0,0,1,0
1,1,0,0,0
1,0,1,0,0
2,1,0,0,0
3,0,1,0,0
3,0,0,1,0
4,0,0,0,0
6,0,0,0,1


That looks better - but we want all the factors combined on one row so we can merge with original dataset. We can use `groupby` and `max` to effectively do a logical OR between the different rows. 

In [7]:
factors_grouped = factors_dummies.groupby(factors_dummies.index).max()
factors_grouped

,A,B,C,D
0,1,1,1,0
1,1,1,0,0
2,1,0,0,0
3,0,1,1,0
4,0,0,0,0
6,1,0,0,1


Final step is to merge the data back together:

In [8]:
factors_merged = df.merge(factors_grouped, how='left', left_index=True, right_index=True)
factors_merged[factors_grouped.columns] = factors_merged[factors_grouped.columns].fillna(0).astype(int)
factors_merged

,LAchildID,Factors,A,B,C,D
0,CHILD1,"A,B,C",1,1,1,0
1,CHILD1,"A,B",1,1,0,0
2,CHILD2,A,1,0,0,0
3,CHILD2,"B, C",0,1,1,0
4,CHILD2,None,0,0,0,0
5,CHILD2,,0,0,0,0
6,CHILD3,"D,A,D",1,0,0,1


We now have dummy variables to simplify further analysis.

The implementation of this functionality can be found in [reports.py](../liiatools/cin_census_pipeline/reports.py). 

The function takes a dataframe and a column name (defaults to "AssessmentFactor") and returns the dataframe with these extra columns added.

In [9]:
from liiatools.cin_census_pipeline.reports import expanded_assessment_factors
expanded_assessment_factors(df, column_name='Factors')

,LAchildID,Factors,A,B,C,D
0,CHILD1,"A,B,C",1,1,1,0
1,CHILD1,"A,B",1,1,0,0
2,CHILD2,A,1,0,0,0
3,CHILD2,"B, C",0,1,1,0
4,CHILD2,None,0,0,0,0
5,CHILD2,,0,0,0,0
6,CHILD3,"D,A,D",1,0,0,1


We can also add a prefix to the dummy columns

In [10]:
expanded_assessment_factors(df, column_name='Factors', prefix="Factors_")

,LAchildID,Factors,Factors_A,Factors_B,Factors_C,Factors_D
0,CHILD1,"A,B,C",1,1,1,0
1,CHILD1,"A,B",1,1,0,0
2,CHILD2,A,1,0,0,0
3,CHILD2,"B, C",0,1,1,0
4,CHILD2,None,0,0,0,0
5,CHILD2,,0,0,0,0
6,CHILD3,"D,A,D",1,0,0,1
